In [60]:
import requests
import pandas as pd
import numpy as np
import import_ipynb
import ChartJS

In [24]:
import argparse
import sys

from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException




def getData(client, customer_id):
    ga_service = client.get_service("GoogleAdsService", version="v5")

    query = """
        SELECT
          campaign.id,
          campaign.name,
          ad_group.id,
          ad_group.name,
          segments.week,
          metrics.active_view_cpm,#Average cost of viewable impressions
          ad_group_criterion.keyword.text,
          ad_group_criterion.keyword.match_type,
          metrics.impressions,
          metrics.clicks,
          metrics.cost_micros,
          metrics.ctr
        FROM keyword_view
        WHERE
          segments.date DURING LAST_MONTH
          AND campaign.advertising_channel_type = 'SEARCH'
          AND ad_group.status = 'ENABLED'
          AND ad_group_criterion.status IN ('ENABLED', 'PAUSED')
        ORDER BY metrics.impressions DESC
        LIMIT 100"""

    # Issues a search request using streaming.
    response = ga_service.search_stream(customer_id, query)
    keyword_match_type_enum = client.get_type(
        "KeywordMatchTypeEnum", version="v5"
    ).KeywordMatchType
    
    all_data=[]
    try:
        for batch in response:
            for row in batch.results:
                single_row = dict()
                single_row['campaign_name'] = row.campaign.name
                single_row['ad_group_name'] = row.ad_group.name
                single_row['keyword'] = row.ad_group_criterion.keyword.text
                single_row['keyword_matchtype'] = row.ad_group_criterion.keyword.match_type
                single_row['impressions'] = row.metrics.impressions
                single_row['clicks'] = row.metrics.clicks
                single_row['cost'] = row.metrics.cost_micros
                print(single_row)
                
                all_data.append(single_row)
    except GoogleAdsException as ex:
        print(
            f'Request with ID "{ex.request_id}" failed with status '
            f'"{ex.error.code().name}" and includes the following errors:'
        )
        for error in ex.failure.errors:
            print(f'\tError with message "{error.message}".')
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"\t\tOn field: {field_path_element.field_name}")
        sys.exit(1)
    df = pd.DataFrame(all_data)
    df.to_csv('google_ads_keywords'.csv)


if __name__ == "__main__":
    # GoogleAdsClient will read the google-ads.yaml configuration file in the
    # home directory if none is specified.
    googleads_client = GoogleAdsClient.load_from_storage(version="v12")

    parser = argparse.ArgumentParser(
        description=("Retrieves a campaign's negative keywords.")
    )
    # The following argument(s) should be provided to run the example.
    parser.add_argument(
        "-c",
        "--customer_id",
        type=str,
        required=True,
        help="The Google Ads customer ID.",
    )
    args = parser.parse_args()

    try:
        main(googleads_client, args.customer_id)
    except GoogleAdsException as ex:
        print(
            f'Request with ID "{ex.request_id}" failed with status '
            f'"{ex.error.code().name}" and includes the following errors:'
        )
        for error in ex.failure.errors:
            print(f'\tError with message "{error.message}".')
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"\t\tOn field: {field_path_element.field_name}")
        sys.exit(1)

In [ ]:
class Main:
    def __init__(self, client, customer_id):
        self.client = client
        self.customer_id = customer_id
        self.columns = ['Campaign_ID', 'Campaign_Name', 'Ad_Group_Name', 'Criterion_ID',
                        'Keyword_Text', 'Match_Type', 'CTR', 'Impressions', 'Clicks' 'Cost_Micros',
                         'Week']
        

        self.df = self.getKeywordText()
        self.charts = self.generateCharts()
        
#     def keyword(self):
#         last = 0
#         orders=pd.DataFrame({})
#         while True:
#             url = 'https://' + self.storeName + '.myshopify.com/admin/api/2022-01/orders.json?limit=250&since_id='+str(last)
#             df = getData(url, {}, self.accessToken)
#             if not 'orders' in df:
#                 break
#             df = pd.json_normalize(df['orders'])
#             orders=pd.concat([orders, df], ignore_index=True)
#             last=df['id'].iloc[-1]
#             if(len(df)<250):
#                 break
#         return orders
    
    def getKeywordText(self):
        df1 = self.keyword()
        df =  df1[self.columns]
        campaign_id = ['1001', '1002', '1003', '1004']
        df['cancel_reason'] = np.random.choice(campaign_id, len(df))
        
        campaign_name = ['Spring Sale', 'Mens Clothing' , 'Winter Clearance', 'Home Decor']
        df["Campaign_Name"] = np.random.choice(source_name, len(df))
        df
        

        Ad_Group_Name =  ["Fashion", "Travel", "Holiday", "Net Banking"]
        df["Ad_Group_Name"] = np.random.choice(Ad_Group_Name, len(df))
        
        Criterion_ID =  ['20001', '20002', '20003', '20004']
            
        df["Criterion_ID"] = np.random.choice(Criterion_ID, len(df))
        
        Keyword_Text =  [
            "Buy designer shoes",
            "Book a luxury hotel in New York",
            "Get cheap flights to Paris",
            "Buy now and save 20%!",
            "Clearance sale - save up to 50%",
            "Free returns and exchanges",
            "Huge selection - shop now",
            "Save big on your favorite brands",
            "24/7 access to your accounts",
            "Sign up for internet banking today"]
        df["Keyword_Text"] = np.random.choice(cities, len(df))
        
        Match_Type = ["BROAD", "EXACT", "PHRASE", "UNKNOWN", "UNSPECIFIED"]
        df["Match_Type"] = np.random.choice(Match_Type, len(df))

        
        df['CTR'] = np.random.uniform(0, 10, df.shape[0])
        df['Impressions'] = np.random.randint(10000, 50000, df.shape[0])
        df['Clicks'] = np.random.randint(1000, 100000, df.shape[0])
        df['Cost_Micros'] = np.random.randint(1000, 10000, df.shape[0])
        
        week = ['7/11/2022', '14/11/2022', '21/11/2022', '28/11/2022']
        df["Week"] = np.random.choice(week, len(df))
        
        return df
        # return custResponse
        
        
    def generateCharts(self):
        df = self.df
        chartsCont = []

        
        #Chart 1
        df['customer.total_spent'] = df['customer.total_spent'].astype(float)
        gk = df.groupby('week').sum().reset_index()
        chart1 = ChartJS.MultiChart(gk['week'], 'Weekly Clicks and Cost per Campaign Over the Last Weeks', 'line', 'It illustrates the relationship between the two metrics over last month.', 
                                   'It can be used to identify trends and make informed decisions about how to allocate advertising resources in the future.It can make sure they are not overspending on their advertising efforts',
                                   '1', gk['Cost_Micros'], gk['Clicks'])
        chartsCont.append(chart1)
        
        
         #Chart 2
        top_keywords = df.sort_values(by='Clicks', ascending=False).head(1)['Keyword_Text']
        df_top = df[df['Keyword_Text'].isin(top_keywords)]
        chart2 = ChartJS.Scatter(df_top['Clicks'], 'Impressions vs Clicks for Top Keyword Text', 'scatter', 'It shows the correlation between Impressions and Clicks, and the extent to which more people seeing a particular keyword text leads to them clicking on it.', 
                                 'It is important to identify any outliers in the data set as they may indicate potential areas of improvement. If the plot shows that a small number of people are seeing the ads but a high percentage of them are clicking on them, you might want to increase the ad budget or expand the targeting to improve performance.',
                                 '2', df_top['Impressions'])
        chartsCont.append(chart2)
        
        #Chart 3
        df1 = df.groupby(['Keyword_Text','week'])['Cost_Micros'].sum().reset_index() 
        top3 = df1['Keyword_Text'].value_counts().head(3).index.tolist() 
        df2 = df1[df1['Keyword_Text'].isin(top3)]
        chart3 = ChartJS.MultiChart(df2['Week'], df2['Keyword_Text'], 'Cost of Keywords by Week', 'col stacked', 'It displays the total cost of each keyword over a period of time, allowing for comparison between different keywords.', 
                                 'It tracks the effectiveness of the customers advertising efforts and makes informed decisions about how to allocate their advertising budget. Look for keywords that have a low CPC but a low number of clicks, and consider adjusting their ad copy or targeting to try and increase the number of clicks they are receiving.',
                                 '3', df2['Cost_Micros'])
        chartsCont.append(chart3)
        
        #Chart 4
        df_grouped = df.groupby('Campaign_Name')['Impressions'].sum().reset_index()
        chart4 = ChartJS.Chart(df_grouped['Campaign_Name'], df_grouped['Impressions'], df2['Keyword_Text'], 'Impressions Distribution by Campaign Name', 'pie', 'It provides a quick visual representation of the relative performance of each campaign.', 
                                 'Focus on the channels that are driving the most impressions and leads.Consider running A/B tests to optimize your campaigns and maximize your ROI.',
                                 '4')
        chartsCont.append(chart4)
        
        #Chart 5
        df_ctr = df[['CTR','Keyword_Text']]
        df_ctr = df_ctr.groupby('Keyword_Text').sum()
        chart5 = ChartJS.Chart(df_ctr['CTR'], df_ctr['Keyword_Text'], 'Distribution of CTR by Keyword Text', 'pie', 'It provides a quick visual representation of the CTR by each keyword Text.', 
                                 'Consider using positive keywords to improve the number of clicks. Also Consider adding more keywords to the mix to increase the overall CTR.',
                                 '5')
        chartsCont.append(chart5)
        
        
        #Chart 6
        grouped_data = df.groupby( ['Campaign_Name']).sum().reset_index()
        chart6 = ChartJS.MultiChart(grouped_data['Campaign_Name'],  'Campaign Performance: Clicks and Cost_Micros', 'col', 'It provides a visual representation of the performance of each campaign in terms of clicks and cost.', 
                                 'Identify underperforming campaigns and consider pausing or discontinuing those campaigns if they are not delivering the results you want in terms of clicks and cost_micros.',
                                 '6', grouped_data['Cost_Micros'], grouped_data['Clicks'])
        chartsCont.append(chart6)
        

    def getCharts(self):
        return self.charts